## GRU model testing: detrended data
Next thing to try is detrended data input for our semi-optimized GRU. Only big difference here will be a rework of SMAPE scoring to take predicted changes, and convert them back into MBD values of SMAPE scoring.

In [ ]:
import shelve
import numpy as np
import matplotlib.pyplot as plt

# Add notebook parent dir to path so we can import from functions/
import sys
sys.path.append('..')

# Import project config file
import config as conf

# Import notebooks specific helper functions
import functions.notebook_helper_functions.notebook15 as funcs
import functions.initialization_functions as init_funcs

# Instantiate paths and model parameters
paths = conf.DataFilePaths()
params = conf.GRU_model_parameters()

# Load data column index
index = shelve.open(paths.PARSED_DATA_COLUMN_INDEX)

# Fire up logger
logger = init_funcs.start_logger(
    logfile = f'{paths.LOG_DIR}/{params.log_file_name}',
    logstart_msg = 'Starting GRU hyperparameter optimization run'
)

OK, so here we go. Hardest part about working with this type of model is getting the data into the correct shape for input. Couple of considerations here:
1. Do training/validation split where data is kept in sequential time order with older data being used for training and newer data being used for validation.
2. Start with only one input feature - the microbusiness density (or detrended microbusiness density).
3. Forecast one point into the future.
4. Be ready to standardize/unstandardize data using statistics from the training set only.
5. Input data is formatted as (batch, timesteps, feature). 

The last part is a little complicated to think about - we have > 3k timeseries with the same time axis, one for each county. We could treat this like 3k features, but I think the better idea is to think of it as one feature and 3k counties * 37 timepoints input datapoints. The trick is, how do we batch/make timesteps out of it? We don't want to present the model timeseries from different counties as if one comes after the next. I think the most obvious way to do this is use a stateless GRU layer and then present each county as a batch. Within that batch we then have the block from our sliding window data parse.

Ok, I think that sounds like as good a place to start as any. Let's take a look at the data:

In [ ]:
# Load data with block size
input_file = f'{paths.PARSED_DATA_PATH}/{params.input_file_root_name}{params.block_size}.npy'
timepoints = np.load(input_file)

# Print some info.
print()
print(f'Input data shape: {timepoints.shape}')
print()

print('Input data column index:')

for column, num in index.items():
    print(f'{column}: {num}')

The dimensions here are:

0. The timepoint block - the size of this axis depends on the width of the block used to scan the data - smaller blocks give more timepoints with num_timepoint_blocks = total_timepoints - block_size + 1. This is also the axis we need to do our training validation split on. First part becomes training, last part becomes validation.
1. The counties - each element here is a county, for the purposes of our first experiment we will treat each county as a batch.
2. The the timepoints in the timepoint block (~row in pandas dataframe).
3. The features (~column in pandas dataframe). To start with, we will work with one feature only - the microbusiness density.

First up - training/validation/testing split, followed by data standardization and then an axis swap to make the data batch major as is the convention for tensorflow:

In [ ]:
datasets = funcs.training_validation_testing_split(
    index,
    timepoints,
    num_counties = params.num_counties,
    input_data_type = params.input_data_type,
    testing_timepoints = params.testing_timepoints,
    training_split_fraction = params.training_split_fraction,
    pad_validation_data = params.pad_validation_data,
    forecast_horizon = params.forecast_horizon
)

datasets, training_mean, training_deviation = funcs.standardize_datasets(datasets)
datasets = funcs.make_batch_major(datasets)

for data_type, data in datasets.items():
    print(f'{data_type} data shape: {data.shape}')

Looks good! Let's build the model. Only additional thing to mention here is that for each time block in the counties, the first n - 5 datapoints are the time ordered input and the last five is the value we are trying to predict. With that in mind, let's go!

In [ ]:
model, history = funcs.train_GRU(
    datasets,
    forecast_horizon = params.forecast_horizon,
    epochs = params.epochs,
    GRU_units = params.GRU_units,
    learning_rate = params.learning_rate,
    save_tensorboard_log = params.save_tensorboard_log,
    tensorboard_log_dir = params.tensorboard_log_dir,
    tensorboard_histogram_freq = params.tensorboard_histogram_freq,
    save_model_checkpoints = params.save_model_checkpoints,
    model_checkpoint_dir = params.model_checkpoint_dir,
    model_checkpoint_threshold = params.model_checkpoint_threshold,
    model_checkpoint_variable = params.model_checkpoint_variable,
    early_stopping = params.early_stopping,
    early_stopping_monitor = params.early_stopping_monitor,
    early_stopping_min_delta = params.early_stopping_min_delta,
    early_stopping_patience = params.early_stopping_patience,
    verbose = params.verbose
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

# Plot and label the training and validation MSE (loss)
ax[0].plot(list(range(len(history.history['loss']))), history.history['loss'], c='black', label='Training Loss')
ax[0].plot(list(range(len(history.history['val_loss']))), history.history['val_loss'], c='red', label='Validation Loss')
 
# Add in a title and axes labels
ax[0].set_title('Mean square error')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')

# Plot and label the training and validation MAE
ax[1].plot(list(range(len(history.history['MAE']))), history.history['MAE'], c='black', label='Training Loss')
ax[1].plot(list(range(len(history.history['val_MAE']))), history.history['val_MAE'], c='red', label='Validation Loss')
 
# Add in a title and axes labels
ax[1].set_title('Mean absolute error')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
 
# Display the plot
plt.legend(loc='best')
plt.show()

In [ ]:
predictions, targets = funcs.make_predictions(
    model, 
    datasets, 
    params.forecast_horizon, 
    training_mean, 
    training_deviation
)

In [ ]:
# Plot predicted vs actual MBD values for validation set

# Set common axis limits
data_pool = list(targets['validation'].flatten()) + list(predictions['GRU_validation'].flatten())
trimmed_data_pool = [i for i in data_pool if i > 0]

plot_max = max(trimmed_data_pool)
plot_min = min(trimmed_data_pool)

# Make plots
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

# Plot with linear axes
ax[0].scatter(targets['validation'].flatten(), predictions['GRU_validation'].flatten(), s=params.plot_point_size, c='red')

# Add y=x line
ax[0].axline((plot_min, plot_min), (plot_max, plot_max), color='black', linestyle='dashed')

# Set axis limits
ax[0].set_xlim([plot_min, plot_max])
ax[0].set_ylim([plot_min, plot_max])

# Add labels
ax[0].set_xlabel('Actual MBD')
ax[0].set_ylabel('Predicted MBD')

# Plot again with log10 axes
ax[1].scatter(targets['validation'].flatten(), predictions['GRU_validation'].flatten(), s=params.plot_point_size, c='red')

# Add y=x line
ax[1].axline((plot_min, plot_min), (plot_max, plot_max), color='black', linestyle='dashed')

# Set axis limits
ax[1].set_xlim([plot_min, plot_max])
ax[1].set_ylim([plot_min, plot_max])

# Add labels
ax[1].set_xlabel('Actual MBD')
ax[1].set_ylabel('Predicted MBD')

# Set log10 scales
ax[1].set_xscale('log')
ax[1].set_yscale('log')

# Add in main title
plt.suptitle('Predicted vs. actual validation microbusiness density')
 
# Display the plot
plt.show()

In [ ]:
# Split off the last three individual month's predictions
# this approximates the final private leaderboard scoring
index_3_validation_predictions = predictions['GRU_validation'][:,2]
index_4_validation_predictions = predictions['GRU_validation'][:,3]
index_5_validation_predictions = predictions['GRU_validation'][:,4]

index_3_validation_targets = targets['validation'][:,2]
index_4_validation_targets = targets['validation'][:,3]
index_5_validation_targets = targets['validation'][:,4]

In [ ]:
# Plot predicted vs actual MBD values for validation set by the forecast horizon index

# Set common axis limits
data_pool = list(targets['validation'].flatten()) + list(predictions['GRU_validation'].flatten())
trimmed_data_pool = [i for i in data_pool if i > 0]

plot_max = max(trimmed_data_pool)
plot_min = min(trimmed_data_pool)

# Make plots
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

# Plot with linear axes
ax[0].scatter(index_3_validation_targets, index_3_validation_predictions, s=params.plot_point_size, label='+3')
ax[0].scatter(index_4_validation_targets, index_4_validation_predictions, s=params.plot_point_size, label='+4')
ax[0].scatter(index_5_validation_targets, index_5_validation_predictions, s=params.plot_point_size, label='+5')

# Add y=x line
ax[0].axline((plot_min, plot_min), (plot_max, plot_max), color='black', linestyle='dashed')

# Set axis limits
ax[0].set_xlim([plot_min, plot_max])
ax[0].set_ylim([plot_min, plot_max])

# Add labels
ax[0].set_xlabel('Actual MBD')
ax[0].set_ylabel('Predicted MBD')

# Plot again with log10 axes
ax[1].scatter(index_3_validation_targets, index_3_validation_predictions, s=params.plot_point_size, label='+3')
ax[1].scatter(index_4_validation_targets, index_4_validation_predictions, s=params.plot_point_size, label='+4')
ax[1].scatter(index_5_validation_targets, index_5_validation_predictions, s=params.plot_point_size, label='+5')

# Add y=x line
ax[1].axline((plot_min, plot_min), (plot_max, plot_max), color='black', linestyle='dashed')

# Set axis limits
ax[1].set_xlim([plot_min, plot_max])
ax[1].set_ylim([plot_min, plot_max])

# Add labels
ax[1].set_xlabel('Actual MBD')
ax[1].set_ylabel('Predicted MBD')

# Set log10 scales
ax[1].set_xscale('log')
ax[1].set_yscale('log')

# Add in a title and axes labels
plt.suptitle('Predicted vs. actual validation microbusiness density')
 
# Display the plot
plt.legend(title='Forecast horizon month', loc='best')
plt.show()

OK, this all looks very fancy and surprisingly good. It's pretty obvious that we have some serious data leakage due to the sliding window used during our data parse. Not sure exactly what to do about that at the moment. For now, i'm going to accept it as a fact of life. This is maybe not so bad - we are not making a model that can predict MBD at an arbitrary time in the future from completely novel data - for the contest, we have all the data we ever will have, we just need to predict 5 month past the end of the dataset. I.e., re-using data that has been trained on already as input for the test set is maybe not so un-realistic. It might be a good idea to adjust our training, validation, testing split such that the targets for the test set have never been seen by the model at all. As it stands now, this is only true of the forecast +5 point. But, from the scatterplot above, it's pretty apparent that we don't do spectacularly worse on that timepoint anyways.

What I would really like to see before I call it a day is some comparison to the naive control. Let's see what we can come up with.

In [ ]:
# Plot predicted vs actual MBD values for validation set by the forecast horizon index

# Set common axis limits
data_pool = list(targets['validation'][2:].flatten()) + list(predictions['GRU_validation'][2:].flatten()) + list(predictions['control_validation'][2:].flatten())
trimmed_data_pool = [i for i in data_pool if i > 0]

plot_max = max(trimmed_data_pool)
plot_min = min(trimmed_data_pool)

# Make plots
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

# Plot with linear axes
ax[0].scatter(targets['validation'][2:], predictions['GRU_validation'][2:], s=params.plot_point_size, label='GRU model')
ax[0].scatter(targets['validation'][2:], predictions['control_validation'][2:], s=params.plot_point_size, label='Naive control')

# Add y=x line
ax[0].axline((plot_min, plot_min), (plot_max, plot_max), color='black', linestyle='dashed')

# Set axis limits
ax[0].set_xlim([plot_min, plot_max])
ax[0].set_ylim([plot_min, plot_max])

# Add labels
ax[0].set_xlabel('Actual MBD')
ax[0].set_ylabel('Predicted MBD')

# Plot again with log10 axes
ax[1].scatter(targets['validation'][2:], predictions['GRU_validation'][2:], s=params.plot_point_size, label='GRU model')
ax[1].scatter(targets['validation'][2:], predictions['control_validation'][2:], s=params.plot_point_size, label='Naive control')

# Add y=x line
ax[1].axline((plot_min, plot_min), (plot_max, plot_max), color='black', linestyle='dashed')

# Set axis limits
ax[1].set_xlim([plot_min, plot_max])
ax[1].set_ylim([plot_min, plot_max])

# Add labels
ax[1].set_xlabel('Actual MBD')
ax[1].set_ylabel('Predicted MBD')

# Set log10 scales
ax[1].set_xscale('log')
ax[1].set_yscale('log')

# Add in a title and axes labels
plt.suptitle('Predicted vs. actual validation microbusiness density: public leaderboard forecast horizon')
 
# Display the plot
plt.legend(loc='best')
plt.show()

OK, we can work with this - not sure I would call it an improvement... but for no hyperparameter optimization, no architecture tuning and no exogenous data, I think we are doing ok... reality is, we are probably going to run out of time for this competition. Let's look at the SMAPE. The major consideration here is that we don't have a true leaderboard smape for prediction in the range [+3,+5] which is what we care about - let's look at just +1 to start. That way we can get a sense of if the control value even makes sense for our bootstrapped control distribution...

In [ ]:
GRU_private_SMAPE_score = funcs.aggregate_SMAPE_score(
    targets['validation'],
    predictions['GRU_validation'],
    [2,3,4]
)

control_private_SMAPE_score = funcs.aggregate_SMAPE_score(
    targets['validation'],
    predictions['control_validation'],
    [2,3,4]
)

GRU_public_SMAPE_score = funcs.aggregate_SMAPE_score(
    targets['validation'],
    predictions['GRU_validation'],
    [0]
)

control_public_SMAPE_score = funcs.aggregate_SMAPE_score(
    targets['validation'],
    predictions['control_validation'],
    [0]
)

print()
print(f'GRU public SMAPE score: {GRU_public_SMAPE_score}')
print(f'Control public SMAPE score: {control_public_SMAPE_score}')
print()
print(f'GRU private SMAPE score: {GRU_private_SMAPE_score}')
print(f'Control private SMAPE score: {control_private_SMAPE_score}')

OK, so the SMAPE scores seem pretty high - especially the control public SMAPE score. We have some experience with that and I would expect something closer to between 1 and 2. Not 5. In fact...

**EDIT**: Fixed it, turns out predictions were not being properly un-standardized. Now we get about a 50:50 shot of observing a SMAPE score as good or better than the aggregate SMAPE on our validation batches with the control.

In [ ]:
from statistics import NormalDist

smape_mean = 1.562
smape_std = 0.5200

print(f'Mean: {smape_mean}')
print(f'Standard deviation: {smape_std}')

leaderboard_score_probability = NormalDist(mu=smape_mean, sigma=smape_std).cdf(1.4609987048968616)
print(f'Probability of SMAPE > 1.46: {1 - leaderboard_score_probability}')

Using the mean and standard deviation from the control bootstrapping SMAPE score distribution (see notebook #11) we get a probability on the order of 3 in a trillion to draw a sample with a SMAPE score of ~5.15 or higher. I would not rely on that probability number too heavily, but suffice to say a SMAPE score of 5 is just about a 7 sigma event and is therefore pretty unusual given our bootstrapping results. Let's double check our control forecast, reshaping etc. and SMAPE scoring. First thought I have is that we are combining SMAPE scores across timepoints - the (100/n) factor in the SMAPE calculation should handle this, i.e having more points should not inflate the SMAPE.

Even after fixing the standardization issue that was causing the above headache, I think I would still like to see the individual SMAPE scores for each timepoint in the validation set.

### TODO
1. Confidence in/improvement to our validation/testing data sets.
2. A good few rounds of GRU optimization
3. More features!

In regards to #1, I think the best way to improve the test set is to ensure that all timepoints in the forecast horizon have never been part of the training or validation data sets. In fact, I think we could simplify everything and preserve more data for training if we just made this a requirement for the validation dataset. To do this, we need to pad the training/validation split by the forecast horizon, so that the first validation target region occurs after the last training target region. We will loose some timepoints doing this, but I think that it's worth it - at least to try.

OK, set up optional padding for the validation set and an optional reserved test set. Feeling pretty good about that. But, it occurs to me that we don't really have a good way to do resampling for bootstrapping. I have two ideas here:
1. Move the validation set around in time - this would give us a few different splits for which we could train a few different models.
2. Look at counties x blocks as one giant pool and draw training/validation samples randomly from that.

Number two is interesting, it would increase our computational efficiency considerably because we could have more control over the batch size, but that would remove our present requirement that each batch be from the same timepoint. I would also be much harder to mitigate data leakage. We would either have to re-parse the data at a frequency that matches the forecast horizon ensuring that parts of forecast regions used for training don't end up in validation forecast regions. Even more stringent would be to parse into non-overlapping blocks and then sample from those..,


There are actually quite a few possibilities here in terms of ways to do the parsing/blocking and then batching/resampling. I don't think we have time to rigorously test all of them. Also, we are testing blind to a degree here, since we won't see true private leaderboard scores for [+3, +5] predictions until it's too late to make changes.

We are going to need to use some intuition here. I think it is important to preserve the time axis within a batch, i.e. we shouldn't mix timeseries from multiple counties in a batch. This limits things quite a bit and makes me feel ok about how we are doing it now. One possible change is to make each county part of the training, validation and testing data and resample that way. But, I feel like that's not the right approach because our samples will reflect the variability between counties, not across timepoints. This limits our ability to bootstrap because the only way to resample is to move the validation set around within the the timeseries, and I don't think chopping the middle of the time series out for validation and leaving the training set with a discontinuity is a great idea either.

On possible saving grace here is that we can generate a resampling distribution for our control model and then ask the question, what is the probability that our model's validation score was drawn from that distribution? If we are really significantly beating the control, we should be able to pick it up that way. Ok, so stay the course and start tuning the GRU model. First three things I want to look at are data block size, GRU units and learning rate. Before we do that, let's get refactored and set up to train multiple models in parallel with tensorboard logging enabled. This will save us a good deal of time in the long run.